In [1]:
%pip install pandas 
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests
%pip install docxtpl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

from typing import Dict


from models.econometrics import Cost, Currency

import math
from typing import Callable
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic

Currency.CLP exchange ratios load status: True


In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Currencies values

In [4]:
import requests # type: ignore
import json
#currency values

# national units https://mindicador.cl/api
query_factor:requests.Response = requests.get('https://mindicador.cl/api',timeout=1000)
ratios_cl = json.loads(query_factor.text)
usd_clp:float = ratios_cl['dolar']['valor']
Cost.set_exchange(Currency.CLP,usd_clp)
Cost.set_exchange(Currency.UF,usd_clp/ratios_cl['uf']['valor']) # 1 dolar in UF
Cost.set_exchange(Currency.UTM,usd_clp/ratios_cl['utm']['valor']) # 1 dolar in Utm

# exchange rates https://app.freecurrencyapi.com/dashboard
query_exchange:requests.Response = requests.get(config["CURRENCY_API_KEY"],timeout=1000)

currency_ratios = json.loads(query_exchange.text)
Cost.set_exchange(Currency.EUR,currency_ratios['data']['EUR'])
Cost.set_exchange(Currency.GBP,currency_ratios['data']['GBP'])
Cost.set_exchange(Currency.BRL,currency_ratios['data']['BRL'])


set $1 Peso Chileno... on USD$0.00 
set $1 UF............. on USD$39.27 
set $1 UTM............ on USD$68.71 
set $1 Euro........... on USD$1.07 
set $1 Pound.......... on USD$1.25 
set $1 Real........... on USD$0.20 


## Address and general info🌎

In [5]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

project:Project = Project(
    title='Sistema Fotovoltaico OnGrid-Mixto',
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu,\nSanto Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )

# psr_sanenrique:Project = Project(
#     title='Sistema Fotovoltaico OnGrid-Mixto PSR San Enrique, Santo Domingo',
#     building = Building(
#         geolocation=GeoPosition(latitude=-33.9081421,longitude=-71.7159192),
#         name='Posta de Salud Rural de San Enrique',
#         address='Ruta G-866 km 0 sn, Localidad San Enrique, Santo Domingo',
#         city='Santo Domingo'),
#     technology= [Tech.PHOTOVOLTAIC]
#     )



api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.861702&longitude=-71.657839&start=20230101&end=20230102&format=JSON


In [6]:
project.weather.get_data().sample(10)

,date,T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M,year,month,day,hour
15,2023-01-01 10:00:00,21.73,1004.83,89.65,0.1,20.71,4.78,2023,01,01,10
5,2023-01-01 00:00:00,14.20,0.00,0.00,None,None,2.73,2023,01,01,00
38,2023-01-02 09:00:00,20.14,916.37,79.07,0.12,32.62,5.38,2023,01,02,09
29,2023-01-02 00:00:00,13.66,0.00,0.00,None,None,6.16,2023,01,02,00
1,2022-12-31 20:00:00,15.62,0.00,0.00,None,None,1.69,2022,12,31,20
27,2023-01-01 22:00:00,14.33,0.00,0.00,None,None,6.82,2023,01,01,22
44,2023-01-02 15:00:00,20.38,983.55,73.39,0.13,49.14,7.27,2023,01,02,15
46,2023-01-02 17:00:00,18.27,788.40,49.95,0.14,73.64,7.77,2023,01,02,17
41,2023-01-02 12:00:00,21.83,981.53,110.48,0.12,14.29,6.80,2023,01,02,12
6,2023-01-01 01:00:00,13.96,0.00,0.00,None,None,2.96,2023,01,01,01


# Integration 

## Energy Consumptions🔌

In [7]:
from models.consumption import ElectricityBill, Energetic
project.building.add_consumptions(description="main",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])
project.building.add_consumptions(description="sec",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [8]:
project.building.consumptions['main'].forecast()
project.building.consumption_forecast(['main','sec'])

boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450
boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450
boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450


,month,energy
0,1,994.0
1,2,900.0
2,3,950.0
3,4,950.0
4,5,1040.0
5,6,1308.0
6,7,1640.0
7,8,1369.0
8,9,1233.5
9,10,1098.0


## Machine Components 🎡

In [9]:
#cspell: disable

#clean previous load
project.components = {}
from IPython.display import clear_output

from models.photovoltaic import Length, PvTechnicalSheet
clear_output(wait=True)

# assembly projects components
project.add_component(
    'generación',
    Photovoltaic(project.weather,'Módulo Primario A',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                quantity=5,
                cost=None,
                orientation=Orientation(15,270-16.46),
                technical_sheet=PvTechnicalSheet(power=655,area=(130.3,238.4,Length.CM)),
                ),
    Photovoltaic(project.weather,'Módulo Secundario B',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                quantity=5,
                cost=None,
                orientation=Orientation(15,16.46),
                technical_sheet=PvTechnicalSheet(power=655,area=(130.3,238.4,Length.CM)),
                )
    )

project.add_component(
    'auxiliar',
    Component('inversor de 7kW',
                model='Voltronic 7kW 48V',
                reference='https://solarbex.com/comprar/inversor-hibrido-7kw-48v-voltronic/',
                specification='Híbrido 7kW Monofásico',
                cost_per_unit=Cost(1255,Currency.EUR)),
    Component('Controlador de Carga',
                model='Enertik 80A 12/24V',
                specification='1000W to 2000 24V output',
                cost_per_unit=Cost(101600,Currency.CLP)),
    Component('medidor bidireccional',
                model='Elster AS1440',
                specification='Trifasico',
                cost_per_unit=Cost(306990,Currency.CLP)),
    Component('almacenamiento',
                model='MP NP12-200',
                specification='Betería 12V 200Ah',
                reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-200ah.html',
                cost_per_unit=Cost(204889,Currency.CLP),
                quantity=10,
                )
    )

project.add_component(
    'obras',
    Component(description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=100,
            ),
    Component(description='estructura de soporte',
            specification='elementos fijación',
            cost_per_unit=Cost((4*800+11400/4+4*700+4000+500)),
            quantity=24
            )
)


Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [10]:
# psr_bucalemu.components['generación'][1].nominal_power()
#pv_log_model(1500) #clp/w

bucket_list = project.bucket_list(Currency.CLP)
    
#unitary price comparison
# print(
#     'estimado',bucket_list['cost'],
#     'on grid',pv_log_model_on_grid(655*10)*655*10,
#     'off grid',pv_log_model_off_grid(655*10)*655*10,
#     'linear',pv_linear(655*10)*655*10)

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [11]:
# psr_bucalemu.get_energy_generation('generación')
project.energy_production('generación')#[['month','day','System_capacity_KW']].groupby(['month','day'],as_index=False).sum()

,Temperature_cell,IRR_incident,System_capacity_KW,date UTC,month,day,hour
0,NaN,NaN,NaN,2022-12-31 19:00:00,12,31,19
1,NaN,NaN,NaN,2022-12-31 20:00:00,12,31,20
2,NaN,NaN,NaN,2022-12-31 21:00:00,12,31,21
3,NaN,NaN,NaN,2022-12-31 22:00:00,12,31,22
4,NaN,NaN,NaN,2022-12-31 23:00:00,12,31,23
5,NaN,NaN,NaN,2023-01-01 00:00:00,1,1,0
6,NaN,NaN,NaN,2023-01-01 01:00:00,1,1,1
7,NaN,NaN,NaN,2023-01-01 02:00:00,1,1,2
8,NaN,NaN,NaN,2023-01-01 03:00:00,1,1,3
9,NaN,NaN,NaN,2023-01-01 04:00:00,1,1,4


In [12]:
from models.emission import emission

emission.annual_projection(2024)
emission.annual_avg()

project.performance(consumptions=['main'],generation_group='generación')

boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

# Write Document 📒


In [ ]:
from docxtpl import * 


In [ ]:
#assembly proxy object 
ctx:dict[str,any] = {
    #about this project
    "project_name": project.title,
    "project_type" : project.technology[0],
    "project_size":"*test",
    "size_unit":"kW",
    "total_cost": project.bucket_list(Currency.CLP)["cost"],
    #site
    "building_name" : project.building.name,
    "city": project.building.city
}

In [ ]:
from docxtpl import DocxTemplate
doc = DocxTemplate("templates/memory_template.docx")
doc.render(ctx)
doc.save(f"build/reporte_{project.building.city}_{project.building.name}.docx")

project.get_context('generación')